In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import datetime
from time import sleep


In [28]:
def queryBicipublica2(biciID):
    try:
        url = 'http://epok.buenosaires.gob.ar/getObjectContent/?id=estaciones_de_bicicletas%7C' + biciID
        response = urllib.urlopen(url)
        rawData = response.read().decode("utf-8")
        parsedData = json.loads(rawData)

        bicisDisponibles = int(parsedData['contenido'][3]['valor'])
        posicionesDisponibles = int(parsedData['contenido'][4]['valor'])
        totales = int(parsedData['contenido'][5]['valor'])
    
    except:
        bicisDisponibles = np.nan
        posicionesDisponibles = np.nan
        totales = np.nan
        
    
    return {'bicisDisponibles':bicisDisponibles,'posicionesDisponibles':posicionesDisponibles,'totales':totales}

In [2]:
#cambiar a http://epok.buenosaires.gob.ar/getObjectContent/?id=estaciones_de_bicicletas%7C2
#sacar todos los datos y calcular en la base despues

def queryBicipublica(biciID, indicador = 'disponibilidad'):
    try:
        
        url = 'http://epok.buenosaires.gob.ar/getObjectContent/?callback=jQuery19108223016225484279_1505305148763&id=estaciones_de_bicicletas%7C' + biciID
        response = urllib.urlopen(url)
        rawData = response.read().decode("utf-8")
        parsedData = json.loads(rawData.split('(',1)[1][:-1])


        bicisDisponibles = int(parsedData['contenido'][3]['valor'])
        posicionesDisponibles = int(parsedData['contenido'][4]['valor'])
        totales = int(parsedData['contenido'][5]['valor'])

        rotas = 100 - ((bicisDisponibles + posicionesDisponibles)*1. / totales * 100)
        #disponibilidad = bicisDisponibles *1. /  totales * 100
        disponibilidad = bicisDisponibles *1. /  (bicisDisponibles + posicionesDisponibles) * 100
        if disponibilidad > 100.0:
            disponibilidad = 100.0
    except:
        rotas = np.nan
        disponibilidad = np.nan
        
    
    if indicador == 'rotas':
        return rotas
    elif indicador == 'ambas':
        return {'rotas':rotas,'disponibilidad':disponibilidad}
    else:
        return disponibilidad

In [3]:
#base madre
geometria = pd.read_csv('../data/estaciones-de-bicicletas-publicas.csv',sep=';')
#convertir en texto los id de las estaciones para el query
geometria.NRO_EST = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in geometria.NRO_EST]
#remover columnas innecesarias
geometria = geometria.loc[:,['X','Y','NOMBRE','NRO_EST']]
geometria.head()



,X,Y,NOMBRE,NRO_EST
0,-58.392452,-34.583133,FACULTAD DE DERECHO,01
1,-58.374822,-34.592589,RETIRO,02
2,-58.368918,-34.611242,ADUANA,03
3,-58.368950,-34.601721,PLAZA ROMA,04
4,-58.369360,-34.627947,PARQUE LEZAMA,06


In [ ]:
#modulo tiempo NUEVOOOO
hora = datetime.datetime.now().hour
#poner un dia limite
diaLimite = 19

fechas = []
bicisDisponibles = []
posicionesDisponibles = []
totales = []
biciIDs = []



#inicia el loop para cargar data
#cambiar hora 10a 0 y hour por day
while True:
    if datetime.datetime.now().hour == 0:
        print 'hora inicio'
        #mientras el dia actual sea menor al limite
        while datetime.datetime.now().day < diaLimite:
            
            fecha = datetime.datetime.now()
            print 'fecha:',fecha
            #inicia el proceso para cada numero de estacion
            for biciID in geometria.NRO_EST:
                #agrego la id de la bici a la lista
                biciIDs.append(biciID)
                #creo el diccionario con datos
                diccio = queryBicipublica2(biciID)
                #extraigo disponibles y total
                bicisDisponibles.append(diccio['bicisDisponibles'])
                posicionesDisponibles.append(diccio['posicionesDisponibles'])
                totales.append(diccio['totales'])
                
                #agrego fecha
                fechas.append(fecha)
                
            #espera 15 min
            print fecha
            sleep(60*15)
        break
    else:
        sleep(60)

datos = pd.DataFrame({'NRO_EST':biciIDs,
                      'fecha':fechas,
                      'bicisDisponibles':bicisDisponibles,
                      'posicionesDisponibles':posicionesDisponibles,
                      'totales':totales
                      })

In [ ]:
#modulo tiempo
hora = datetime.datetime.now().hour
#poner un dia limite
diaLimite = 19

fechas = []
disponibilidades = []
roturas = []
biciIDs = []

#inicia el loop para cargar data
#cambiar hora 10a 0 y hour por day
while True:
    if datetime.datetime.now().hour == 0:
        print 'hora inicio'
        #mientras el dia actual sea menor al limite
        while datetime.datetime.now().day < diaLimite:
            
            fecha = datetime.datetime.now()
            print 'fecha:',fecha
            #inicia el proceso para cada numero de estacion
            for biciID in geometria.NRO_EST:
                #agrego la id de la bici a la lista
                biciIDs.append(biciID)
                #agrego la disponibilidad de la estacion a la lista
                disponibilidades.append(queryBicipublica(biciID, indicador = 'disponibilidad'))
                #agrego fecha
                fechas.append(fecha)
                
            #espera 15 min
            print fecha
            sleep(60*15)
        break
    else:
        sleep(60)

datos = pd.DataFrame({'NRO_EST':biciIDs,
                      'fecha':fechas,
                      'disponibilidad':disponibilidades#,
                      #'roturas':roturas
                      })

In [ ]:
print 'datos', datos.shape
print 'geometria',geometria.shape

In [ ]:
data = geometria.merge(datos)
print data.shape
data.head()

In [ ]:
pwd

In [ ]:
data.to_csv('../data/data18sept.csv')

In [ ]:
import geopandas as gpd

In [ ]:
barrios = gpd.read_file('../data/barrios-rar/barrios_badata.shp')
barrios = barrios.to_crs(epsg=4326)
barrios.head()

In [ ]:
biciShape = gpd.read_file('../data/estaciones-de-bicicletas-rar/estaciones_de_bicicletas.shp')
biciShape = biciShape.to_crs(epsg=4326)
print biciShape.shape
biciShape.head()

In [ ]:
bicisConBarrio = gpd.sjoin(biciShape, barrios, how="inner", op='intersects')
print bicisConBarrio.shape
print bicisConBarrio.BARRIO.isnull().sum()

In [ ]:
datos.head()

In [ ]:
biciShape.NRO_EST = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in biciShape.NRO_EST]


In [ ]:
biciShape = biciShape.merge(datos)
biciShape.shape

In [ ]:
biciShape.fecha = biciShape.fecha.map(str)

In [ ]:
biciShape.to_file('../data/18septiembre')